In [1]:
import pandas as pd
import numpy as np


In [2]:
raw_df = pd.read_csv('data/weatherAUS.csv')
raw_df.dropna(subset=['RainTomorrow'], inplace=True)

In [19]:
# 1. Convert RainTomorrow to boolean
raw_df['RainTomorrow_bool'] = (raw_df['RainTomorrow'] == 'Yes').astype(int)

# 2. Percentage of RainTomorrow = Yes
rain_percentage = raw_df.groupby('Location')['RainTomorrow_bool'].mean() * 100

# 3. Compute averages of features split by RainTomorrow
features = ['Humidity3pm', 'Pressure3pm', 'MinTemp', 'MaxTemp']
averages = (
    raw_df.groupby(['Location', 'RainTomorrow'])[features]
      .mean()
      .unstack()   # Creates separate Yes/No columns
)

# Rename columns for clarity
averages.columns = [f"{col}_{val}" for col, val in averages.columns]

# 4. Combine everything into one DataFrame
result = pd.concat([rain_percentage, averages], axis=1).reset_index()

# Rename RainTomorrow percentage column
result.rename(columns={'RainTomorrow_bool': 'RainTomorrow_Yes_Percentage'}, inplace=True)

# Round for readability
result = result.round(2)

# Compute totals across all locations
total_rain_percentage = raw_df['RainTomorrow_bool'].mean() * 100
total_avgs = (
    raw_df.groupby('RainTomorrow')[features]
      .mean()
      .unstack()
)

# Flatten columns
total_avgs.index = [f"{col}_{val}" for col, val in total_avgs.index]

# Combine into single Series
total_row = pd.concat([pd.Series({'Location': 'TOTAL',
                                  'RainTomorrow_Yes_Percentage': total_rain_percentage}),
                       total_avgs])

# Append to result
result_with_total = pd.concat([result, total_row.to_frame().T], ignore_index=True)

# Round values
result_with_total = result_with_total.round(2)

print(result_with_total.head())

        Location RainTomorrow_Yes_Percentage Humidity3pm_No Humidity3pm_Yes  \
0       Adelaide                       22.27          40.26            60.7   
1         Albany                       29.91          65.43            72.0   
2         Albury                       20.52          43.17           66.23   
3   AliceSprings                        8.05           21.4           54.74   
4  BadgerysCreek                       19.91          46.57           71.14   

  Pressure3pm_No Pressure3pm_Yes MinTemp_No MinTemp_Yes MaxTemp_No MaxTemp_Yes  
0        1017.99         1012.53      12.73       12.27      23.93        19.5  
1        1018.17         1012.61      13.34       12.04      20.57       18.92  
2        1016.85         1011.48       9.29       10.42      23.42       19.58  
3        1013.23         1008.94      12.64       18.64      29.31       28.48  
4        1015.83         1014.85      10.51       13.67       24.3       22.91  


In [20]:
raw_df.Pressure3pm.describe()

count    128212.000000
mean       1015.258204
std           7.036677
min         977.100000
25%        1010.400000
50%        1015.200000
75%        1020.000000
max        1039.600000
Name: Pressure3pm, dtype: float64

In [21]:
raw_df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow', 'RainTomorrow_code',
       'RainToday_code', 'RainTomorrow_bool'],
      dtype='object')